<a href="https://colab.research.google.com/github/rahulpandey89/Age_and_Gender_Prediction/blob/master/custom_ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -U spacy -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 39.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.7.1 requires spacy<3.8.0,>=3.7.2, but you have spacy 3.8.4 which is incompatible.


In [2]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [3]:
import json
f = open('/content/training_data.json')
TRAIN_DATA = json.load(f)

In [4]:
for text, annot in tqdm(TRAIN_DATA['annotations']):
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 5/5 [00:00<00:00, 141.21it/s]


In [5]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [6]:
! python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     45.05    3.70    2.78    5.56    0.04
 78     200        443.16   1255.92  100.00  100.00  100.00    1.00
176     400          0.00      0.00  100.00  100.00  100.00    1.00
276     600          0.00      0.00  100.00  100.00  100.00    1.00
459     800          0.00      0.00  100.00  100.00  100.00    1.00
659    1000          0.00      0.00  100.00  100.00  100.00    1.00
859    1200          0.00      0.00  100.00  100.00  100.00    1.00
1059    1400          0.00      0.00  100.00  100.00  100.00    1.00
1259    1600          0.00      0.0

In [7]:
nlp_ner = spacy.load("/content/model-best")

In [8]:
doc=nlp_ner(''' According to the CoinMarketCap website, $Trump has a total market valuation of about $12bn (£9.8bn), while $Melania's stands at around $1.7bn.
Follow live updates on inauguration day
Trump had previously called crypto a "scam" but during the 2024 election campaign became the first presidential candidate to accept digital assets as donations.
During the campaign, his family launched a cryptocurrency company called World Liberty Financial - which aims to lead "a financial revolution by dismantling the stranglehold of traditional financial institutions", and is also selling a crypto coin.
The new Trump coin was launched from Trump Organization affiliate CIC Digital LLC, which is linked to previous sales of crypto collectable NFTs launched in 2022 that made millions of dollars but have since fallen dramatically in value for their owners.
According to CoinGecko, the NFTs once sold for more than $1,000 but have since dropped in value to around $300.''')

In [9]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter